# 0. Installing Transformers and Importing Dependencies

In [1]:
!pip install transformers

In [2]:
from transformers import pipeline
from bs4 import BeautifulSoup
import requests

# 1. Load Summarization Pipeline

In [5]:
summarizer = pipeline("summarization", model="google/flan-t5-small")

C:\Users\Dell\AppData\Roaming\Python\Python311\site-packages\huggingface_hub\file_download.py:133: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Dell\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


d:\anaconda3\Lib\site-packages\keras\src\initializers\initializers.py:120: UserWarning: The initializer RandomNormal is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initializer instance more than once.
  warnings.warn(
All PyTorch model weights were used when initializing TFT5ForConditionalGeneration.

All the weights of TFT5ForConditionalGeneration were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


# 2. Get Blog Post from Medium

In [6]:
URL = "https://medium.com/@ai-club/ml101-introduction-to-the-world-of-machine-learning-2-86e32c740b58"

In [8]:
r = requests.get(URL)

In [9]:
soup = BeautifulSoup(r.text, 'html.parser')
results = soup.find_all(['h1', 'p'])
text = [result.text for result in results]
ARTICLE = ' '.join(text)

In [10]:
ARTICLE

'Sign up Sign in Sign up Sign in ML101: Introduction to the world of Machine Learning — 2 PTU AI CLUB Follow -- Listen Share Authors: Harsha, Sakti Priya V, Yuvarani VD and Apurva Introduction Our last article in the ML101 series by PTU AI CLUB introduced you to the world of machine learning and its infinite use cases. In this article we will be diving a little deeper into this world by exploring the types of machine learning and the ethical aspects related to machine learning. We will also be providing you with a roadmap for getting started in this world. Enough talk, let’s ML! Types of Machine Learning Based on the methods and way of learning, machine learning is divided into mainly four types, which are: 1. Supervised Machine Learning2. Unsupervised Machine Learning3. Semi-supervised Machine Learning4. Reinforcement Learning Supervised machine learning is based on supervision. It means in the supervised learning technique, we train the machines using the “labeled” dataset, and based

# 3. Chunk Text

In [11]:
max_chunk = 500

In [9]:
ARTICLE = ARTICLE.replace('.', '.<eos>')
ARTICLE = ARTICLE.replace('?', '?<eos>')
ARTICLE = ARTICLE.replace('!', '!<eos>')

In [10]:
sentences = ARTICLE.split('<eos>')
current_chunk = 0 
chunks = []
for sentence in sentences:
    if len(chunks) == current_chunk + 1: 
        if len(chunks[current_chunk]) + len(sentence.split(' ')) <= max_chunk:
            chunks[current_chunk].extend(sentence.split(' '))
        else:
            current_chunk += 1
            chunks.append(sentence.split(' '))
    else:
        print(current_chunk)
        chunks.append(sentence.split(' '))

for chunk_id in range(len(chunks)):
    chunks[chunk_id] = ' '.join(chunks[chunk_id])

0


In [11]:
len(chunks)

1

# 4. Summarize Text

In [12]:
res = summarizer(chunks, max_length=120, min_length=30, do_sample=False)

Your max_length is set to 120, but your input_length is only 29. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=14)


In [13]:
res[0]

{'summary_text': 'Crypto, Markets, Trading Receive Stories from @TK or is This Just The Beginning? @TK, markets, trading, crypto, markets, trading .'}

In [14]:
' '.join([summ['summary_text'] for summ in res])

'Crypto, Markets, Trading Receive Stories from @TK or is This Just The Beginning? @TK, markets, trading, crypto, markets, trading .'

In [15]:
text = ' '.join([summ['summary_text'] for summ in res])

# 5. Output to Text File

In [16]:
text

'Crypto, Markets, Trading Receive Stories from @TK or is This Just The Beginning? @TK, markets, trading, crypto, markets, trading .'

In [17]:
with open('blogsummary.txt', 'w') as f:
    f.write(text)